## 预训练模型实验

In [ ]:
import time
from torch.utils.tensorboard import SummaryWriter
import os
import surpport.mySQL as mySQL
import surpport.myfunction as MF
from surpport.Args import Args
import torch.nn.functional as F
import torch
import numpy as np
from surpport.nnstructure import Classifier7

### sup


In [ ]:
def recorder_build(arg):
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    return recorder


def train(arg, model, recorder, opt, tr_dataloader, ts_dataloader, writer):
    # train
    for epoch in range(1, arg.epoch_num+1):
        # print(epoch)
        recorder[str(epoch)+'-th'] = dict()
        rd = recorder[str(epoch)+'-th']
        # 由于图经过拼合，所以需要多一个dataloader的过程
        # 前两个是list
        tr_loss, tr_acc = MF.train(
            epoch,
            model, opt, tr_dataloader,
            arg,
            writer
        )
        mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

        el_loss, el_acc, logits, labels = MF.evaluate(
            epoch,
            model, opt, ts_dataloader,
            arg,
            writer
        )

        mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
        mySQL.rcd_result(logits, labels, rd)

        val_acc = np.mean(el_acc, axis=0)
        MF.save_best(val_acc, model, arg)

        mySQL.save_final(epoch, model, val_acc, arg, opt)

    mySQL.save_recorder(recorder, arg, 'flow')
    print('best acc: %.4f' % (arg.best_acc))
    return None


### test

In [3]:
heads = [3, 6, 12, 24]
alphas = 9
nums = [2, 6, 10, 14]
pairs = []
for id, v in enumerate(heads):
    pairs.append((v, 9, nums[id]))

In [ ]:
from surpport.Args import Args
def rec_arg(arg: Args, p):
    arg.pip_num = p[0]
    arg.display_freq = 50
    arg.exception_acc = 0.18
    return arg

In [ ]:
from surpport.cross_Dprocess import c_dataload

count = 0
for p in pairs:
    arg = rec_arg(Args(), p)
    model = Classifier7(arg) # 都不接受参数，所以没问题
    checkpoint = torch.load(dir)
    model.load_state_dict(checkpoint['net'])
    model = model.cuda()
    arg.d_prepare('Cross', 2)
    # ! 每次运行都改
    count += 1
    arg.m_info(m_name='m7', m_task='220913_exp', num=count,)
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    writer = SummaryWriter(arg.tar_path+'\\Journal')

    opt_arg = {'params': model.parameters()}
    #    'lr': 6e-5, 'eps': 1e-8, 'weight_decay': 0.1}
    opt = torch.optim.Adam(**opt_arg)

    arg, tr_dataloader, ts_dataloader = c_dataload(arg, model, opt, p[1])

    train(arg, model, recorder, opt, tr_dataloader, ts_dataloader, writer)
    writer.close()